In [1]:
%load_ext autoreload
%autoreload 2
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [2]:
from varian_models import *
from scipy.optimize import minimize
import fastcat as fc
bcca_smaller = np.load('/home/xcite/MV_CBCT/log_kv_phantom_over_air_776x776.npy')[15:-15]

# Looking at the one slice

There is still a fudge factor of 1.03 on the projection to account for fastcats 0.97 I guess.

Also a fundge factor for the dose right now.

And Catphan_Teflon unrevised whatever that is

In [3]:
np.random.seed(0)
angles = np.linspace(np.pi/2,np.pi*2,2)      
kv_catphan = Catphan_404_kV()
kv_catphan.get_proj(angles,fudge=0.3,ASG=True)

(19, 19, 19) 19
Initializing filter bowtie3
   Scatter is filtered by bowtie
Initializing ASG
Running Simulations
    Simulating 10 keV
    Simulating 20 keV
    Simulating 30 keV
    Simulating 40 keV
    Simulating 50 keV
    Simulating 60 keV
    Simulating 70 keV
    Simulating 80 keV
    Simulating 90 keV
    Simulating 100 keV
Weighting simulations


In [4]:
plt.figure()
# plt.plot(np.mean(bcca_smaller[445,195:225],0)/np.std(bcca_smaller[425,195:225,:],0))
arr_av = np.mean(bcca_smaller[445,195:225],0)
arr_std = np.std(bcca_smaller[425,195:225,:],0)
snr = arr_av/arr_std
plt.plot([np.mean(snr[ii:ii+4]) for ii in range(0,len(snr),4)],c='darkorange')

# arr_av = np.mean((np.roll((kv_catphan.proj[0]),-3)*1.03)[20:50,:],0)
# arr_std = np.std((np.roll((kv_catphan.proj[0]),-3)*1.03)[20:50,:],0)
arr_av = np.mean((np.roll((kv_catphan.proj[0]),-3))[20:50,:],0)
arr_std = np.std((np.roll((kv_catphan.proj[0]),-3))[20:50,:],0)
snr = arr_av/arr_std
plt.plot([np.mean(snr[ii:ii+4]) for ii in range(0,len(snr),4)],c='cornflowerblue')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets look at all of them at once

In [39]:
angles = np.load('/home/xcite/MV_CBCT/angles_kv.npy')
projs_bcca = np.zeros([887,64,512])
kv_catphan = Catphan_404_kV()
projs_bcca[:,17:-17,:] =  bcca_smaller[:,195:225,:]

CNR_fc = []
CNR_exp = []

im_fc = []
im_exp = []

dose = 0.11# 0.233 # 0.244 #*3

fudge = dose*1.43

CNR_result_fc = []
CNR_result_exp = []

np.random.seed(0)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

for ii in range(4,0,-1):
    angles_temp = angles[::ii]
    kv_catphan.geomet.nVoxel = np.array([10,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    %time kv_catphan.get_proj(angles_temp,fudge=fudge,ASG=True) 
    kv_catphan.proj[:,:17,:] = 0 # Matching the images
    kv_catphan.proj[:,-17:,:] = 0
    kv_catphan.geomet.nVoxel = np.array([256,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    kv_catphan.reconstruct('FDK',filt='ram_lak')
    vol_fc = kv_catphan.img
    
    kv_catphan.proj = 10*projs_bcca[::ii].astype(np.float32)
    kv_catphan.angles = angles[::ii][-1::-1]
    kv_catphan.reconstruct('FDK',filt='ram_lak')
    vol_exp = kv_catphan.img

    contrast_exp2 = []
    contrast_fc2 = []

    for ii in range(120,136):

        contrast_exp2.append(return_CNR_exp2(vol_exp[ii],True))
        contrast_fc2.append(return_CNR_fc2(vol_fc[ii],True))
    
    contrast_bcca = np.mean(contrast_exp2,0)
    contrast_fc = np.mean(contrast_fc2,0)

    reses = []
    rem = []

    for off in np.arange(-0.2,0.2,0.01):
        for off2 in np.arange(-0.2,0.2,0.01):
            reses.append(minimize(HU_fun,[off,contrast_fc[-3]+off2],args=(contrast_fc,contrast_bcca)))
            rem.append(reses[-1].fun)

    res = reses[np.argmin(rem)]

    fc_data = (contrast_fc - res.x[0])*(res.x[1])
    vol_fc_adj = (vol_fc - res.x[0])*(res.x[1])
    
    CNR_exp4 = []
    CNR_fc4 = []

    for ii in range(120,136):

        CNR_exp4.append(return_CNR_exp2(vol_exp[ii]))
        CNR_fc4.append(return_CNR_fc2(vol_fc_adj[ii]))
    
    CNR_result_fc.append((np.mean(CNR_fc4),np.std(np.mean(CNR_fc4,1))))
    CNR_result_exp.append((np.mean(CNR_exp4),np.std(np.mean(CNR_exp4,1))))

(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 90.28640444925418 number of photons 0.6831236943135264
CPU times: user 18.5 s, sys: 908 ms, total: 19.4 s
Wall time: 19.5 s
bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 90.28637037902128 number of photons 0.6831236943135264
CPU times: user 24.4 s, sys: 1.22 s, total: 25.6 s
Wall time: 25.7 s
bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 90.28640043506405 number of photons 0.6831236943135264
CPU times: user 35.5 s, sys: 1.6 s, total: 37.1 s
Wall time: 37.1 s
bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 90.2864254434734 number of photons 0.6831236943135264
CPU times: user 1min 8s, sys: 2.81 s, total: 1min 11s
Wall time: 1min 11s


In [38]:
plt.figure()
# plt.plot(kv_catphan.s.x,kv_catphan.s.y)
plt.plot(kv_catphan.s.x,kv_catphan.s.y)
# x = np.load('data/energies.npy')
# y = np.load('data/spec_100.npy') # don't calculate too slow
plt.plot(x,y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
np.save('CNR_kv_exp',CNR_exp4)
np.save('CNR_kv_fc',CNR_fc4)
# np.load('CNR_kv_exp',np.mean(CNR_exp4,0))
# np.save('CNR_kv_fc',np.mean(CNR_fc4,0))


x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[4,7])
plt.subplot(211)
plt.title('CNR Comparison')
plt.errorbar(x,np.mean(CNR_exp4,0),np.std(CNR_exp4,0),fmt='s',color='darkorange',label='')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc4,0),np.std(CNR_fc4,0),fmt='*',color='cornflowerblue',label='fastCAT')
plt.grid()
plt.ylabel('CNR')
# plt.xlabel('Relative Electron Density')
# plt.subplot(212)
# plt.errorbar(x,np.mean(CNR_exp3,0),np.std(CNR_exp3,0),fmt='s',color='darkorange',label='')
# plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc3,0),np.std(CNR_fc3,0),fmt='*',color='cornflowerblue',label='fastCAT')
# plt.grid()
# plt.ylabel('CNR')
# plt.xlabel('Relative Electron Density')
# plt.tight_layout()
# plt.savefig('Comparison_CNR.png',dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'CNR')

In [21]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = 120 #self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind],vmin=0,vmax=0.5,cmap='gray')
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

fig, ax = plt.subplots(1, 1)

# X = np.array(projections.T[:,:,5:])
X = np.array(vol_fc).T

tracker = IndexTracker(ax, X)

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [176]:
CNR_err_fc = np.std(CNR_fc3)
CNR_err_exp = np.std(CNR_exp3)

plt.figure()
plt.errorbar([1,2,3,4],[CNR_result_fc[ii][0] for ii in range(4)],fmt='b*',yerr =  [CNR_result_fc[ii][1] for ii in range(4)])
plt.errorbar([1.1,2.1,3.1,4.1],[CNR_result_exp[ii][0] for ii in range(4)],fmt='r*',yerr = [CNR_result_exp[ii][1] for ii in range(4)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [177]:
np.save('CNR_fc_by_dose',[CNR_result_fc[ii][0] for ii in range(4)])
np.save('CNR_err_fc_by_dose',[CNR_result_fc[ii][1] for ii in range(4)])
np.save('CNR_exp_by_dose',[CNR_result_exp[ii][0] for ii in range(4)])
np.save('CNR_err_exp_by_dose',[CNR_result_exp[ii][1] for ii in range(4)])

# This is the one for the gos

In [40]:
from glob import glob

In [41]:
CNR_fc = []
CNR_exp = []

im_fc = []
im_exp = []

fudge_2 = 1 # 0.97 
dose = 0.11
fudge = dose/0.717*1.61#*4.5

CNR_result_fc = []
CNR_result_exp = []

angles = np.load('/home/xcite/MV_CBCT/angles_6x.npy')
exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_6x_300_*.npy'))[np.array([1,3,0,2])]

np.random.seed(0)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

kv_catphan = Catphan_404_6x()

(19, 41, 41) 19


In [42]:
for ii in range(4,-1,-1):
    
    angles_temp = angles[::ii]
    print(len(angles_temp))
    kv_catphan.geomet.nVoxel = np.array([10,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    %time kv_catphan.get_proj(angles_temp,fudge=fudge) 
    kv_catphan.geomet.nVoxel = np.array([256,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    kv_catphan.reconstruct('FDK',filt='ram_lak')
    
    vol_fc = kv_catphan.img
    vol_exp = np.rot90(np.load(exp_files[ii-1]).T,0)

    contrast_exp2 = []
    contrast_fc2 = []

    for ii in range(20):

        contrast_exp2.append(return_CNR_exp3(vol_exp[25 +ii],True))
        contrast_fc2.append(return_CNR_fc3(vol_fc[120+ii],True))
    
    contrast_bcca = np.mean(contrast_exp2,0)
    contrast_fc = np.mean(contrast_fc2,0)

    reses = []
    rem = []

    for off in np.arange(-0.2,0.2,0.01):
        for off2 in np.arange(-0.2,0.2,0.01):
            reses.append(minimize(HU_fun,[off,contrast_fc[-3]+off2],args=(contrast_fc,contrast_bcca)))
            rem.append(reses[-1].fun)

    res = reses[np.argmin(rem)]

    vol_fc_adj = (vol_fc - res.x[0])*(res.x[1])
    fc_data = (contrast_fc - res.x[0])*(res.x[1])
    
    plt.figure()
    plt.plot(fc_data,'b*')
    plt.plot(contrast_bcca,'r*')
    
    CNR_fc3 = []
    CNR_exp3 = []
    
    for ii in range(20):

        CNR_exp3.append(return_CNR_exp3(vol_exp[25 + ii]))
        CNR_fc3.append(return_CNR_fc3(vol_fc_adj[120+ii]))
    
    plt.figure()
    plt.plot(np.mean(CNR_exp3,0),'r*')
    plt.plot(np.mean(CNR_fc3,0),'b*')
    CNR_result_fc.append((np.mean(CNR_fc3),np.std(np.mean(CNR_fc3,1))))
    CNR_result_exp.append((np.mean(CNR_exp3),np.std(np.mean(CNR_exp3,1))))

124
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 2042.5315407826688 number of photons 0.026371213427188782
CPU times: user 19.2 s, sys: 821 ms, total: 20 s
Wall time: 20.1 s


/home/xcite/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

165
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 2042.5213154010728 number of photons 0.026371213427188782
CPU times: user 25.2 s, sys: 1.12 s, total: 26.4 s
Wall time: 26.4 s


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

493
bowtie is off False
Using the bowtie filter True
really doing it!


KeyboardInterrupt: 

AttributeError: 'Catphan_404_6x' object has no attribute 'astra_recon'

In [100]:
np.save('CNR_6x_exp',CNR_exp3)
np.save('CNR_6x_fc',CNR_fc3)

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure()
plt.errorbar(x,np.mean(CNR_exp3,0),np.std(CNR_exp3,0),fmt='s',color='darkorange',label='')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc3,0),np.std(CNR_fc3,0),fmt='*',color='cornflowerblue',label='fastCAT')
plt.grid()
plt.ylabel('CNR')
plt.xlabel('Relative Electron Density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Relative Electron Density')

In [19]:
nviews = 180

s = fc.Spectrum()
s.load('Varian_truebeam_phasespace')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
# phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=0.0)

(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is None number of photons 0.010763158668836276


In [22]:
X = np.array(phantom.proj)

plt.figure()
for ii in range(X.shape[0]):
    
    imshow(np.rot90(X[ii, :, :]),cmap='gray',vmax = 16,vmin=0,aspect='auto')        
#     plt.ylabel('projection %s' % ii)
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'pngs/{ii}.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [274]:
CNR_err_fc = np.std(CNR_fc3)
CNR_err_exp = np.std(CNR_exp3)

plt.figure()
plt.errorbar([1,2,3,4],[CNR_result_fc[ii][0] for ii in range(3,-1,-1)],fmt='b*',yerr =  [CNR_result_fc[ii][1] for ii in range(4)])
plt.errorbar([1.1,2.1,3.1,4.1],[CNR_result_exp[ii][0] for ii in range(4)],fmt='r*',yerr = [CNR_result_exp[ii][1] for ii in range(4)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [275]:
# np.save('CNR_fc_by_dose_kv',[CNR_result_fc[ii][0] for ii in range(4)])
# np.save('CNR_err_fc_by_dose_kv',[CNR_result_fc[ii][1] for ii in range(4)])
# np.save('CNR_exp_by_dose_kv',[CNR_result_exp[ii][0] for ii in range(4)])
# np.save('CNR_err_exp_by_dose_kv',[CNR_result_exp[ii][1] for ii in range(4)])

In [5]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [6]:
fc = np.load('CNR_fc_by_dose_kv.npy')
fc_err = np.load('CNR_err_fc_by_dose_kv.npy')
exp = np.load('CNR_exp_by_dose_kv.npy')
exp_err = np.load('CNR_err_exp_by_dose_kv.npy')

dose_lab = np.array(np.round([300,300/2,300/3,300/4],1))[-1::-1]
dose = np.array([6,4,2,0])[-1::-1]
plt.figure(figsize=[8,7],dpi=300)
plt.subplot(224)
plt.bar(dose,exp[-1::-1],yerr=exp_err,width=0.4,color='darkorange')
plt.bar(dose+0.5,fc,yerr=fc_err,width=0.4,color='cornflowerblue')
plt.grid('on')
plt.xticks(ticks=dose,labels= dose_lab)
plt.xlabel('MU')
plt.ylabel('Average CNR')
plt.legend(['Varian 6 MV','Fastcat 6 MV'],loc=2)
plt.ylim([0,14])

print('Mv',np.max(np.abs(fc-exp[-1::-1])))
print('Mv mean',np.mean(np.abs(fc-exp[-1::-1])/exp[-1::-1]))
# circle1 = plt.text(160,-20,'Experimental',color='k',size='x-large');ax.add_artist(circle1)

fc = np.load('CNR_fc_by_dose.npy')
fc_err = np.load('CNR_err_fc_by_dose.npy')
exp = np.load('CNR_exp_by_dose.npy')
exp_err = np.load('CNR_err_exp_by_dose.npy')

dose_lab = np.array(np.round([21.1,21.1/2,21.1/3,21.1/4],1))[-1::-1]
plt.subplot(222)
plt.title('CNR vs Dose')
plt.bar(dose,exp,yerr=exp_err,width=0.4,color='darkorange')
plt.bar(dose+0.5,fc,yerr=fc_err,width=0.4,color='cornflowerblue')
plt.grid('on')
plt.xticks(ticks=dose,labels= dose_lab)
plt.xlabel('CTDI$_{vol}$')
plt.ylabel('Average CNR')
plt.legend(['Varian OBI 100 kVp','Fastcat 100 kVp'],loc=2)
plt.ylim([0,20])
# plt.tight_layout()
# plt.savefig('Comparison_dose_CNR.png',dpi=300)

CNR_exp4 = np.load('CNR_kv_exp.npy')
CNR_fc4 = np.load('CNR_kv_fc.npy')

print('kv',np.max(np.abs(fc-exp)))
print('kv mean',np.mean(np.abs(fc-exp)/exp))

x = [.853,0.945,.998,1.147,1.36,1.87]

# plt.figure(figsize=[4,7])
ax = plt.subplot(221)
plt.title('CNR Comparison')
plt.errorbar(x,np.mean(CNR_exp4,0),np.std(CNR_exp4,0),fmt='s',color='darkorange',label='Varian OBI 100 kVp')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc4,0),np.std(CNR_fc4,0),fmt='*',color='cornflowerblue',label='Fastcat 100 kVp')
plt.ylim([5.5,13.5])
plt.grid()
plt.legend()
plt.ylabel('CNR')

print('kV',np.mean(CNR_exp4,0),np.mean(CNR_fc4,0))
circle1 = plt.text(0.55,9.5,'100 kVp',color='k',size='x-large',rotation = 90,va='center',ha='center');ax.add_artist(circle1)
CNR_exp4 = np.load('CNR_6x_exp.npy')
CNR_fc4 = np.load('CNR_6x_fc.npy')

print('MV',np.mean(CNR_exp4,0),np.mean(CNR_fc4,0))
ax = plt.subplot(223)
plt.errorbar(x,np.mean(CNR_exp4,0),np.std(CNR_exp4,0),fmt='s',color='darkorange',label='Varian 6 MV')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc4,0),np.std(CNR_fc4,0),fmt='*',color='cornflowerblue',label='Fastcat 6 MV')
plt.grid()
plt.ylim([5.5,13.5])
plt.xlabel('Relative Electron Density')
plt.ylabel('CNR')
plt.legend(loc=2)
circle1 = plt.text(0.55,9.5,'6 MV',color='k',size='x-large',rotation = 90,va='center',ha='center');ax.add_artist(circle1)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mv 0.27184242627271615
Mv mean 0.01224674119812932
kv 0.2585394970017969
kv mean 0.011865205733436426
kV [ 6.13627562  6.65418818  6.96017875  7.94255765  8.92661792 12.0334916 ] [ 6.52715027  6.86852884  7.15360037  7.86648842  9.11832678 12.02601766]
MV [ 6.22099712  6.72520099  7.01377286  7.81545882  8.78833733 10.99384381] [ 6.35354648  6.88275005  7.10808886  7.80658702  8.86848638 10.81812807]


In [3]:
print(fc,exp)

[ 7.87694855  9.12412549 11.45808587 15.63366093] [ 7.91022679  9.29294846 11.35792347 15.89220043]


In [82]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

from scipy.optimize import minimize

reses = []
rem = []

for off in np.arange(-0.2,0.2,0.01):
    reses.append(minimize(HU_fun,[off,contrast_fc[-3]+0.12],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)

res = reses[np.argmin(rem)]

fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (im_fc[-1] - res.x[0])*(res.x[1])

kv_catphan.img[64] = fc_image
x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure()
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
# plt.figure()
# plt.plot(CNR_exp[0],'r*')
# plt.plot(kv_catphan.return_CNR(),'b*')
# new_CNR_fc.append(return_CNR(fc_image,im)[1])    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
np.save('fastcat_images_kv_worst-best',im_fc)
np.save('exp_images_kv_worst-best',im_exp)
np.save('CNR_fc_kv',CNR_fc_kv)
np.save('CNR_exp_kv',CNR_exp)
np.save('vol_fc_kv_adj',vol_fc_adj)
np.save('vol_exp_kv',vol_exp)

In [374]:
plt.imsave('im_for_imagej.png',np.hstack((im_fc[0],im_exp[0])),cmap='gray')
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'


In [44]:
CNR_fc = np.load('CNR_fc_kv.npy')
CNR_exp = np.load('CNR_exp_kv.npy')

In [378]:
plt.imsave('im_for_imagej.png',np.hstack((fc_image,recon_slice)),cmap='gray')
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'


In [67]:
from scipy.optimize import minimize
CNR_fc = np.load('CNR_fc_kv.npy')
CNR_exp = np.load('CNR_exp_kv.npy')

im_fc = np.load('fastcat_images_kv_worst-best.npy')
im_exp = np.load('exp_images_kv_worst-best.npy')

new_CNR_fc = []

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

def return_CNR_fc(proj,return_contrast = False):

    im = create_mask(proj.shape,r=5.75,radius=0.4)
    contrast_fc, CNR_fc, noise_fc = return_CNR(proj,im)
#         print(contrast_fc ,'fastcat')
    if return_contrast:
        return contrast_fc
    else:
        return CNR_fc

def return_CNR_exp(proj,return_contrast = False):

    recon_slice = (proj)
    im = create_mask(recon_slice.shape,r=5.85,off = [-0.15,-0.02],rot = 0.7,radius=0.40)
    contrast_fc, CNR_fc, noise_fc = return_CNR(recon_slice,im)
#         print(contrast_fc, 'exp')
    if return_contrast:
        return contrast_fc
    else:
        return CNR_fc

im = create_mask(im_fc.shape,r=5.75,radius=0.4)

for ii in range(3,-1,-1):
    
    contrast_fc = return_CNR_fc(im_fc[ii],True) 
    contrast_bcca = return_CNR_exp(im_exp[ii],True) 
    
    if ii == 3:
        reses = []
        rem = []

        for off in np.arange(-0.2,0.2,0.01):
            reses.append(minimize(HU_fun,[off,contrast_fc[-3]-0.01],args=(contrast_fc,contrast_bcca)))
            rem.append(reses[-1].fun)

        res = reses[np.argmin(rem)]

    fc_data = (contrast_fc - res.x[0])*(res.x[1])
    fc_image = (im_fc[ii] - res.x[0])*(res.x[1])
    
    new_CNR_fc.append(return_CNR(fc_image,im)[1])
    
    x = [.853,0.945,.998,1.147,1.36,1.87]

    plt.figure(figsize=[8,8])
    plt.subplot(221)
    plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
    plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
    plt.legend()
    plt.subplot(222)
    plt.plot(recon_slice[256,:],c='darkorange')
    plt.plot(np.rot90(fc_image,0)[256,:],c='cornflowerblue')
    plt.subplot(223)
    plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.13,vmax=0.35)
    plt.subplot(224)
    plt.imshow(np.rot90(recon_slice,0),cmap='gray',vmin=0.13,vmax=0.35)

IndexError: index 3 is out of bounds for axis 0 with size 1

In [68]:
plt.figure()
plt.plot((new_CNR_fc[3]),'b*')
plt.plot(CNR_exp[3],'r*')
# plt.figure()
# plt.plot(contrast_fc,'b*')
# plt.plot(contrast_bcca,'r*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: list index out of range

In [218]:
res

      fun: 0.0026893317699432373
 hess_inv: array([[0.01104779, 0.01104779],
       [0.01104779, 0.14830004]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 81
      nit: 4
     njev: 27
   status: 0
  success: True
        x: array([-0.03572898,  0.82787716])

In [197]:
plt.figure()
plt.plot(CNR_fc[3],'b*')
plt.plot(CNR_exp[3],'r*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [172]:
plt.figure()
plt.imshow(np.rot90(im_exp[0],-1))
# plt.imshow(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [196]:
plt.figure()
for ii in range(4):
    plt.plot(ii,np.mean(CNR_fc[ii],0),'r*')
    plt.plot(ii,np.mean(CNR_exp[ii],0),'b*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
recon_slice = np.rot90(kv_catphan.img[64],3)
im = create_mask(recon_slice.shape,r=5.85,radius=0.45,off = [-0.15,-0.02],rot = 0.7)
contrast_exp, CNR_exp, noise_fc = return_CNR(recon_slice,im,show_map=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
print(contrast_exp)

[0.16167541 0.17474125 0.18429291 0.20837197 0.24231511 0.3306908 ]


In [ ]:
# bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kV_full.npy').T #recon_kv_jan_2021.npy')
# recon_slice = np.rot90(np.mean(bcca_recon.T[39:40],0)).copy()
# im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
# contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in np.arange(-0.2,0.2,0.01):
    reses.append(minimize(HU_fun,[contrast_fc[2],contrast_fc[-3]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]
    
fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (phantom.img[4] - res.x[0])*(res.x[1])

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()
plt.subplot(222)
plt.plot(recon_slice[256,:],c='darkorange')
plt.plot(np.rot90(fc_image,0)[256,:],c='cornflowerblue')
plt.subplot(223)
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)
plt.subplot(224)
plt.imshow(np.rot90(recon_slice,2),cmap='gray',vmin=0.013,vmax=0.027)